# Preliminary Notes

This script was developed to calculate hydrometeor particles for the doctoral dissertation of **Sari (2025)**, titled *A Study of Hailstorms in Maritime Tropical Area: Surabaya Hail Events, Indonesia*, at the University of Illinois Urbana–Champaign. The code has been applied in that study, and portions of it are already published in *JGR: Atmospheres* as *Hailstorm Events Over a Maritime Tropical Region: Storm Environments and Characteristics* (Sari & Lasher-Trapp, 2025).

---

All rights to this code are reserved by **Sari (2025)**.

## WRF - Hail Diameter Calculation (Gamma-Distribution Approach)
Originally, this code was adapted from the WRF Model, where it was implemented in Fortran under the file name `module_diag_nwp.f90`.  For more flexibility, the author reimplemented it in Python with relevant modifications, including the treatment of density, number thresholds, and other parameters.

### Purpose
Compute hailstone diameters throughout the model column and extract:
- **HMAXK1**: maximum hail diameter at the **lowest** model level  
- **HMAX2D**: maximum hail diameter **anywhere in the column**  
- **Full vertical profile** of hail diameter at each grid point  

---

### Inputs
Arrays with shape `(time, zh, yh, xh)`:
- `qhl`, `qg` — hail and graupel **mixing ratios** $[\mathrm{kg}/\mathrm{kg}]$  
- `chl`, `chw` — hail and graupel **number concentrations** $[\#/\mathrm{kg}]$  
- $\rho$ — **dry-air density** $[\mathrm{kg}\,\mathrm{m}^{-3}]$  

---

### Core Idea
Total hail-phase **mass** and **number** are computed by combining hail and graupel and converting to volume densities:

$$
q_h^{\ast} = (q_{hl}+q_g)\,\rho, \qquad n_h^{\ast} = (c_{hl}+c_{hw})\,\rho
$$

A **gamma size distribution** is assumed for hail:

$$
N(D) = N_0\,D^{\mu}\,e^{-\lambda D}
$$

with parameters

$$
\lambda = \left(\frac{a\,n_h^{\ast}}{q_h^{\ast}}\right)^{1/b}, \qquad 
a=\frac{\pi}{6}\rho_h, \quad b=3, \quad \mu=0
$$

where $\rho_h$ is hail density (default $=900~\mathrm{kg}\,\mathrm{m}^{-3}$).

---

### Discretization & Search
- **51 logarithmic bins** spanning **0.5 mm** to **7.5 cm**.  
- Geometric midpoints $D_g$ and widths $\Delta D$ define discrete counts.  
- A cumulative number integral is computed from large to small diameters.  
- The hail size is taken where the cumulative number first exceeds a threshold:  

$$
\sum N(D_g)\,\Delta D \;\ge\; \texttt{thresh\_conc}
$$

- Default threshold: $thresh\_conc = 3 \times 10^{-4}$

If the crossing occurs between bins, **linear interpolation** refines the diameter.

---

### Outputs
- `hail_profile` $(time, zh, yh, xh)$: hail diameter at each vertical level and grid cell  
- `hail_max2d` $(time, yh, xh)$: column-maximum hail diameter (HMAX2D)  
- `hail_maxk1` $(time, yh, xh)$: maximum hail diameter at the lowest level (HMAXK1)  

---

### Constants & Defaults
- Hail bulk density: $\rho_h = 900\ \mathrm{kg}\,\mathrm{m}^{-3}$  
- Gamma parameters: $a=\dfrac{\pi}{6}\rho_h,\; b=3,\; \mu=0$  
- Floors for stability:  
  - $q_h^{\ast} \ge 1\times10^{-12}$  
  - $n_h^{\ast} \ge 1\times10^{-8}$  
- Bin range: $0.5~\mathrm{mm}$ – $7.5~\mathrm{cm}$  

---

### Notes & Assumptions
- Hail and graupel are **combined** into a single rimed-ice category.  
- Threshold logic identifies the **largest representative size**.  
- Output diameters are in **meters**.  
- Triple nested loops are used per time step; for large domains, vectorization or acceleration may be beneficial.  


### Hail Size Calculation Design For CM1 Output

In [1]:
# import module

import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.ticker
import matplotlib.ticker as tick

In [2]:
# Re-import necessary libraries after code environment reset
import numpy as np
from tqdm import tqdm

def calculate_hail_diameter_profile(qhl, chl, qg, chw, rho):
    """
    Calculates hail diameter profile over vertical layers from atmospheric data.

    Parameters:
    - qhl, qg: Mixing ratios of hail and graupel (kg/kg), shape (time, zh, yh, xh)
    - chl, chw: Number concentrations of hail and graupel (#/kg), shape (time, zh, yh, xh)
    - rho: Dry-air density (kg/m^3), shape (time, zh, yh, xh)

    Returns:
    - hail_profile: Hail diameter at each vertical level, shape (time, zh, yh, xh)
    - hail_max2d: Maximum hail size across vertical levels, shape (time, yh, xh)
    - hail_maxk1: Hail size at lowest vertical level, shape (time, yh, xh)
    """
    time, nk, ny, nx = qhl.shape
    hail_profile = np.zeros((time, nk, ny, nx))
    hail_max2d = np.zeros((time, ny, nx))
    hail_maxk1 = np.zeros((time, ny, nx))

    # Constants
    xrho_h = 900.0
    xam_h = np.pi / 6.0 * xrho_h
    xbm_h = 3.0
    xmu_h = 0.0
    thresh_conc = 0.0003 #0.001

    # Bins for hail size
    xxDx = np.zeros(51)
    xxDx[0] = 500e-6
    xxDx[-1] = 0.075
    for n in range(1, 50):
        xxDx[n] = np.exp((n) / 50.0 * np.log(xxDx[-1] / xxDx[0]) + np.log(xxDx[0]))
    xxDg = np.sqrt(xxDx[:-1] * xxDx[1:])
    xdtg = xxDx[1:] - xxDx[:-1]

    # Loop over time with progress bar
    for t in tqdm(range(time), desc="⏳ Calculating hail diameter"):
        temp_qh = np.maximum(1e-12, (qhl[t] + qg[t]) * rho[t])
        temp_nh = np.maximum(1e-8, (chl[t] + chw[t]) * rho[t])

        for k in range(nk):
            for j in range(ny):
                for i in range(nx):
                    if temp_qh[k, j, i] < 1e-6:
                        continue

                    lamh = (xam_h * temp_nh[k, j, i] / temp_qh[k, j, i])**(1.0 / xbm_h)
                    N0_h = temp_nh[k, j, i]

                    sum_nh = 0.0
                    sum_t = 0.0
                    ng = 49
                    while ng >= 0:
                        f_d = N0_h * xxDg[ng]**xmu_h * np.exp(-lamh * xxDg[ng]) * xdtg[ng]
                        sum_nh += f_d
                        if sum_nh > thresh_conc:
                            break
                        sum_t = sum_nh
                        ng -= 1

                    if ng >= 49:
                        hail_max = xxDg[-1]
                    elif ng >= 0 and xxDg[ng + 1] > 1e-3:
                        hail_max = xxDg[ng] - (sum_nh - thresh_conc) / (sum_nh - sum_t) * (xxDg[ng] - xxDg[ng + 1])
                    else:
                        hail_max = 1e-4

                    hail_profile[t, k, j, i] = hail_max  # Store full vertical profile
                    if k == 0:
                        hail_maxk1[t, j, i] = max(hail_maxk1[t, j, i], hail_max)
                    hail_max2d[t, j, i] = max(hail_max2d[t, j, i], hail_max)

    return hail_profile, hail_max2d, hail_maxk1


In [2]:
### Open Data

def open_and_extract_multiple(files, variable_map):
    """
    Opens multiple NetCDF files, extracts, and renames variables.

    Parameters:
    - files (list of str): List of file paths to NetCDF files.
    - variable_map (dict): A dictionary mapping desired variable names to dataset keys.

    Returns:
    - list of dict: A list of dictionaries, each containing renamed variables from a file.
    """
    all_variables = []

    for file_path in files:
        dataset = xr.open_dataset(file_path)
        variables = {new_name: dataset[old_name] for new_name, old_name in variable_map.items()}
        all_variables.append(variables)

    return all_variables


In [8]:
import xarray as xr

# Directory and file paths
dir = '# add your directory here'
files = [
    dir + '# add your netcdf file here', #1

# Variable mapping
variable_map = {
    'xh': 'xh',
    'yh': 'yh',
    'z': 'zf',
    'time': 'time',
    'cp': 'cape',
    'cn': 'cin',
    'u': 'uinterp',
    'v': 'vinterp',
    'w': 'winterp',
    'qv': 'qv',
    'qc': 'qc',
    'qr': 'qr',
    'qg': 'qg',
    'qh': 'qhl',
    'ref': 'dbz', 
    'chw': 'chw',
    'chl': 'chl',
    'crw': 'crw', 
    'th': 'th', 
    'p' : 'prs', 
    'rho' : 'rho'
}

# Open all files into separate datasets
datasets = [xr.open_dataset(file) for file in files]

# Extract and rename variables for each file
all_variables = []
for idx, dataset in enumerate(datasets, start=1):
    variables = {f"{new_name}_{idx}": dataset[old_name] for new_name, old_name in variable_map.items()}
    all_variables.append(variables)

# Unpack variables into individual references
for idx, variables in enumerate(all_variables, start=1):
    globals().update({f"{name}": var for name, var in variables.items()})

### Looping all data calculated

In [5]:
import xarray as xr
import os

# Output folder
output_dir = "# add directory for output"
os.makedirs(output_dir, exist_ok=True)

# Loop over dataset versions (1 to 4, adjust if needed)
# for idx in range(1, 5):
for idx in [12]:
    print(f"Processing set {idx}...")

    # Get each variable dynamically
    qh = globals()[f"qh_{idx}"].isel(time=slice(20, 80)).load()
    qg = globals()[f"qg_{idx}"].isel(time=slice(20, 80)).load()
    chl = globals()[f"chl_{idx}"].isel(time=slice(20, 80)).load()
    chw = globals()[f"chw_{idx}"].isel(time=slice(20, 80)).load()
    rho = globals()[f"rho_{idx}"].isel(time=slice(20, 80)).load()

    # Run the hail diameter calculation
    hmxprf, hmx2d, hmxk1 = calculate_hail_diameter_profile(
        qh.values, chl.values, qg.values, chw.values, rho.values
    )

    # Wrap into DataArrays
    hail_max2d_da = xr.DataArray(
        hmx2d,
        dims=["time", "yh", "xh"],
        coords={"time": qh.time, "yh": qh.yh, "xh": qh.xh},
        name="hail_max2d",
        attrs={
            "long_name": "Maximum hail diameter at any vertical level",
            "units": "m"
        }
    )

    hail_maxk1_da = xr.DataArray(
        hmxk1,
        dims=["time", "yh", "xh"],
        coords={"time": qh.time, "yh": qh.yh, "xh": qh.xh},
        name="hail_maxk1",
        attrs={
            "long_name": "Maximum hail diameter at surface level",
            "units": "m"
        }
    )

    hail_profile_da = xr.DataArray(
        hmxprf,
        dims=["time", "zh", "yh", "xh"],
        coords={"time": qh.time, "zh": qh.zh, "yh": qh.yh, "xh": qh.xh},
        name="hail_diameter_profile",
        attrs={
            "long_name": "Hail diameter at each vertical level",
            "units": "m"
        }
    )

    # Save as NetCDF
    hail_max2d_da.to_netcdf(f"{output_dir}/hmx2d_set{idx}_new_netcape.nc")
    #hail_maxk1_da.to_netcdf(f"{output_dir}/hmxk1_d7_set{idx}_t20t80_tc00003_1.nc")
    hail_profile_da.to_netcdf(f"{output_dir}/hmxprf_set{idx}_new_netcape.nc")

    print(f"Saved hail data for set {idx} ✅")


Processing set 12...


⏳ Calculating hail diameter: 100%|██████████| 60/60 [7:10:34<00:00, 430.57s/it]  


Saved hail data for set 12 ✅


## Hydrometeor Particles Calculation with simple approach

### Mass–Diameter Relationship for Hydrometeors

This method is based on a simple **mass–diameter relationship** using:

$$
m = \frac{q}{n}
$$

> where:
> $m$ = average mass per particle \[kg]
> $q$ = mass mixing ratio \[kg/kg]
> $n$ = number concentration \[#/kg]

---

Assuming **spherical particles**, the volume–mass relation is:

$$
m = \frac{\pi}{6} \rho_p D^3 \quad \Rightarrow \quad D = \left( \frac{6m}{\pi \rho_p} \right)^{1/3}
$$

Where:

* $\rho_p$ = particle material density \[kg/m³]
* $D$ = diameter \[m]

---

### Suggested Hydrometeor Densities

Use realistic values for $\rho_p$ depending on the particle type:

* **Rain**: \~1000 kg/m³
* **Graupel**: \~400–500 kg/m³
* **Hail**: \~700–900 kg/m³

---

### Final Formula (Diameter in meters)

$$
D = \left( \frac{6 \cdot \frac{q}{n}}{\pi \cdot \rho_p} \right)^{1/3}
$$

Or simplified:

$$
D = \left( \frac{6q}{n \pi \rho_p} \right)^{1/3}
$$

$\rho_p$ can be also defined as appropriate for each hydrometeor type.



In [ ]:
import os
import xarray as xr
import numpy as np
from tqdm import tqdm

def compute_d(q, n, rho_p):
    """
    Compute hydrometeor equivalent spherical diameter from 
    mixing ratio and number concentration.

    Parameters:
    - q: Mass mixing ratio of hydrometeor [kg/kg]
    - n: Number concentration of hydrometeor [#/kg]
    - rho_p: Prescribed bulk density of the hydrometeor [kg/m³]

    Returns:
    - D: xarray.DataArray of equivalent spherical diameters [m],
         same dimensions as input q and n

    Notes:
    - Average particle mass is computed as m = q / n.
    - Diameters are estimated assuming spherical particles:
          D = ((6 * m) / (π * ρ_p))^(1/3)
    - NaNs are assigned where n ≤ 0.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        m = xr.where(n > 0, q / n, np.nan)
        D = ((6 * m) / (np.pi * rho_p))**(1/3)
    return D


def calculate_hydrometeor_diameters(ds):
    """
    Calculates particle diameters for rain, graupel, and hail 
    using a simple mass–diameter relationship.

    Parameters:
    - ds: xarray.Dataset containing:
        - qr, nr: Rain mixing ratio (kg/kg) and number concentration (#/kg)
        - qg, ng: Graupel mixing ratio (kg/kg) and number concentration (#/kg)
        - qh, nh: Hail mixing ratio (kg/kg) and number concentration (#/kg)

    Returns:
    - diameter_raindrop: DataArray of raindrop diameters [m], same dims as input
    - diameter_graupel: DataArray of graupel diameters [m], same dims as input
    - diameter_hail: DataArray of hail diameters [m], same dims as input

    Notes:
    - Prescribed bulk densities:
        * Rain = 1000 kg/m³
        * Graupel = 500 kg/m³ or can be also modified use direct grauepl density from CM1 output
        * Hail = 900 kg/m³
    - Computation is performed separately at each time step 
      and concatenated into full time series.
    - Outputs include variable names and metadata (units, long_name).
    """
    # Prescribed hydrometeor densities [kg/m³]
    rho_rain = 1000.0
    rho_graupel = 500.0
    rho_hail = 900.0

    qr = ds["qr"]
    nr = ds["nr"]
    qg = ds["qg"]
    ng = ds["ng"]
    qh = ds["qh"]
    nh = ds["nh"]

    time_dim = ds.sizes["time"]
    diam_rain = []
    diam_graupel = []
    diam_hail = []

    for t in tqdm(range(time_dim), desc="Calculating diameters over time"):
        d_rain = compute_d(qr.isel(time=t), nr.isel(time=t), rho_rain)
        d_grau = compute_d(qg.isel(time=t), ng.isel(time=t), rho_graupel)
        d_hail = compute_d(qh.isel(time=t), nh.isel(time=t), rho_hail)

        diam_rain.append(d_rain)
        diam_graupel.append(d_grau)
        diam_hail.append(d_hail)

    # Stack back into DataArray with time
    rain_d_da = xr.concat(diam_rain, dim="time")
    graupel_d_da = xr.concat(diam_graupel, dim="time")
    hail_d_da = xr.concat(diam_hail, dim="time")

    rain_d_da.name = "diameter_raindrop"
    graupel_d_da.name = "diameter_graupel"
    hail_d_da.name = "diameter_hail"

    rain_d_da.attrs = {'units': 'm', 'long_name': 'Raindrop diameter'}
    graupel_d_da.attrs = {'units': 'm', 'long_name': 'Graupel diameter'}
    hail_d_da.attrs = {'units': 'm', 'long_name': 'Hail diameter'}

    return rain_d_da, graupel_d_da, hail_d_da


In [ ]:
# === Input/Output Setup ===

input_dir = "# add your input directory here"
output_dir = "# add your output directory here"

selected_sets = [2, 12, 11, 8, 9, 10, 5]
file_template = "filtered_w10_new_set{}.nc" #set according your nc files

for i in selected_sets:
    path_in = os.path.join(input_dir, file_template.format(i))
    path_out = os.path.join(output_dir, f"filtered_w10_new_dia_set{i}.nc")

    if os.path.exists(path_in):
        print(f"\nProcessing set {i}...")
        ds = xr.open_dataset(path_in)

        rain_d, graupel_d, hail_d = calculate_hydrometeor_diameters(ds)

        # Merge and save
        ds_out = ds.copy()
        ds_out["diameter_raindrop"] = rain_d
        ds_out["diameter_graupel"] = graupel_d
        ds_out["diameter_hail"] = hail_d

        ds_out.to_netcdf(path_out)
        print(f"Saved: {path_out}")
    else:
        print(f"File not found: {path_in}")

In [ ]:
### Calculate the statistics

import os
import xarray as xr
import numpy as np

# === Input Directory and File Pattern ===
input_dir = "# add your input directory here"
selected_sets = [2, 12, 11, 8, 9, 10, 5]
file_template = "filtered_w10_new_dia_set{}.nc" #set according your nc files

# === Variable Conversion Mapping (name, units, scale factor) ===
variables = {
    "diameter_raindrop": {"label": "Raindrop", "unit": "µm", "scale": 1e6}, # in micro meter
    "diameter_graupel": {"label": "Graupel", "unit": "mm", "scale": 1e3}, # in milimeter
    "diameter_hail": {"label": "Hail", "unit": "mm", "scale": 1e3}, # in milimeter
}

# === Store results grouped by variable
stats_by_var = {var: {} for var in variables}

# === Loop over datasets
for i in selected_sets:
    file_path = os.path.join(input_dir, file_template.format(i))
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    ds = xr.open_dataset(file_path)

    for varname, props in variables.items():
        if varname not in ds:
            print(f"Variable {varname} not found in set {i}")
            continue

        data = ds[varname] * props["scale"]
        vals = data.values.flatten()
        vals = vals[~np.isnan(vals)]

        # Compute statistics
        stats = {
            "max": np.nanmax(vals),
            "p99": np.nanpercentile(vals, 99),
            "p95": np.nanpercentile(vals, 95),
            "median": np.nanmedian(vals),
            "mean": np.nanmean(vals),
        }

        stats_by_var[varname][f"set{i}"] = stats

    ds.close()

# === Print grouped by variable
for varname, set_stats in stats_by_var.items():
    label = variables[varname]["label"]
    unit = variables[varname]["unit"]
    print(f"\n=== {label} Diameter Stats [{unit}] ===")
    print(f"{'Dataset':<8}  {'Max':>8}  {'99th':>8}  {'95th':>8}  {'Median':>8}  {'Mean':>8}")
    print("-" * 55)
    for dataset, stats in set_stats.items():
        print(f"{dataset:<8}  {stats['max']:8.2f}  {stats['p99']:8.2f}  {stats['p95']:8.2f}  "
              f"{stats['median']:8.2f}  {stats['mean']:8.2f}")
